In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --u (--upgrade, --upgrade-strategy, --use-deprecated, --use-feature, --use-pep517, --user?)


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def load_dataset():
    ds = []
    for root, dirs, files in os.walk('CIC dataset/'):
        for file in files:            
            ds.append(pd.read_csv(os.path.join(root,file)))
    return ds

In [ ]:
def preprocess_data(datas):    
    #merging data
    data = pd.concat(datas, axis = 0, ignore_index = True)
    print(f"Before duplicate removal{data.shape}")

    #remove leading or trailing whitespace from col names
    col_names = {col: col.strip() for col in data.columns}
    data.rename(columns = col_names, inplace = True)
    
    #duplicate rows removal
    data = data.drop_duplicates(keep = 'first')
    
    #duplicate columns removal
    columns = data.columns
    identical_columns =[]
    list_control = columns.copy().tolist()
    for col1 in columns:
        for col2 in columns:
            if(col1!=col2):
                if(data[col1].equals(data[col2])):
                    if(col2 not in identical_columns and col2 in list_control):
                        identical_columns.append(col2)
                        if col1 in list_control:
                            list_control.remove(col1)
                        if col2 in list_control: 
                            list_control.remove(col2)
                    elif(col2 in identical_columns and col2 in list_control):
                        if col2 in list_control: 
                            list_control.remove(col2)    
    for col in identical_columns:
        data.drop(columns = col, inplace = True)
                    
    print(f"After duplicate removal{data.shape}")

    #removing rows with missing values
    missing_rows = data.isna().any(axis=1).sum()
    print(f'\nTotal rows with missing values: {missing_rows}')
    data = data.dropna()
    nan_count = data.isnull().sum().sum()
    print(f"Total NaN values: {nan_count}")
    
    # Treating infinite values
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    print(f"After missing value rows' removal{data.shape}")
    
    #splitting data
    target = data['Label']
    features = data.drop('Label',axis = 1)

    
    return features,target

In [ ]:
def encode_data(data):
    encoder = LabelEncoder()
    for col in data.columns:
        if(data[col].dtype == 'object'):
            data[col] = encoder.fit_transform(data[col])
    return ds

In [ ]:
def standardize_data(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

In [ ]:
def oversample(data,target):
    sampler = RandomOverSampler(sampling_strategy = 'minority')
    new_data,new_target = sampler.fit_resample(data,target)
    return new_data, new_target

In [ ]:
def dimensionality_reduction(data):    
    pca = PCA(n_components = 10)
    data = pca.fit_transform(data)

In [8]:
ds = load_dataset()

NameError: name 'load_dataset' is not defined

In [ ]:
features, target = preprocess_data(ds)


Before duplicate removal(2830743, 79)
After duplicate removal(2522362, 67)

Total rows with missing values: 353
Total NaN values: 0
After missing value rows' removal(2522009, 67)


In [ ]:
#convert categorical values to numerical
features = encode_data(features)


In [ ]:
features = standardize_data(features)


In [ ]:
features, target = oversample(features, target)


In [ ]:
features = dimensionality_reduction(features)
features = np.expand_dims(features, axis=-1)
print(ds)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
target = np.array(target)
model = Sequential([
    Dense(128, activation='relu', input_shape=(features.shape[1],)), 
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(np.unique(target)), activation='softmax') 
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()